In [1]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Activation, GlobalAveragePooling2D, BatchNormalization, Dropout, Flatten
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

import wandb
from wandb.keras import WandbCallback
from keras_contrib.layers import GroupNormalization

from src.freezegroupnorm import FreezeGroupNormalization

Using TensorFlow backend.


In [1]:
MODULE = None
NAME = None
BS = None
GS = None
LR = None

In [ ]:
normalization_module = {
    'batchnorm': BatchNormalization,
    'groupnorm': GroupNormalization,
    'freezegroupnorm': FreezeGroupNormalization 
}

In [ ]:
configs = {
    'module': MODULE,
    'batch_size': BS,
    'group_size': GS,
    'learning_rate': LR
}

wandb.init(
    project='Group Normalization CIFAR-10',
    name=NAME,
    config=configs
)

config = wandb.config

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

img_width=32
img_height=32

X_train = X_train.astype('float32')
X_train /= 255.
X_test = X_test.astype('float32')
X_test /= 255.

#reshape input data
X_train = X_train.reshape(X_train.shape[0], img_width, img_height, 3)
X_test = X_test.reshape(X_test.shape[0], img_width, img_height, 3)

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
it_train = datagen.flow(X_train, y_train, batch_size=config.batch_size)
steps = int(X_train.shape[0] / config.batch_size)

In [ ]:
opt = SGD(lr=config.learning_rate, momentum=0.9)

# build model
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
model.add(normalization_module[config.module]())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(normalization_module[config.module]())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(normalization_module[config.module]())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(normalization_module[config.module]())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(normalization_module[config.module]())
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(normalization_module[config.module]())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(normalization_module[config.module]())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
model.fit_generator(it_train, steps_per_epoch=steps, validation_data=(X_test, y_test), epochs=50, batch_size=config.batch_size,
    callbacks=[WandbCallback()])